In [ ]:
#importing necessary libraries

import numpy as np
import imageio
import os
import cv2
import tifffile as tiff
import math
import random

In [ ]:
#defining necessary constants

patch_size = 320                    #size of patches the image is broken doen into
datasize = 6000                     #total no, of patches to be created
train_data = 5000                   #no. of patches to be used for training 

In [ ]:
#function to resize the images such that there are an integral no. of patches in them

def resize(x,patch_size=patch_size):
    img_height = x.shape[0]
    img_width = x.shape[1]
    n_channels = x.shape[2]
    
    # make extended img so that it contains integer number of patches
    npatches_vertical = math.ceil(img_height / patch_size)
    npatches_horizontal = math.ceil(img_width / patch_size)
    extended_height = patch_size * npatches_vertical
    extended_width = patch_size * npatches_horizontal
    ext_x = np.full((extended_height, extended_width, n_channels) ,255 , dtype=np.float32)
    
    # fill extended image with mirrors
    ext_x[:img_height, :img_width, :] = x
    for i in range(img_height, extended_height):
        ext_x[i, :, :] = ext_x[2 * img_height - i - 1, :, :]
    for j in range(img_width, extended_width):
        ext_x[:, j, :] = ext_x[:, 2 * img_width - j - 1, :]

    # now we assemble all patches in one array
    return ext_x

In [ ]:
#function to obtain an (m x n) array (containing the respective class of the pixel) from rgb images of shape (m x n x 3)

def get_classes_from_image(Y):
  
  colours={
    0:[255,255,255],                                    #white  ---> no class
    1:[0,0,0],                                          #black  ---> roads
    2:[100,100,100],                                    #grey   ---> buildings
    3:[0,0,150],                                        #dark blue  ---> water
    4:[0,255,0],                                        #light green  ---> grass
    5:[255,255,0],                                      #yellow   ---> rails
    6:[0,125,0],                                        #dark green  ---> trees
    7:[150,150,255],                                    #light blue  --->pools
    8:[150,80,0]                                        #brown  --->bare soil
    }
  
  temp=np.zeros((Y.shape[0],Y.shape[1]))
  for j in range(Y.shape[0]):
    for k in range(Y.shape[1]):
      for l in range(9):
        if colours[l]==list(Y[j,k]):
          temp[j,k]=l
          
  return temp

In [ ]:
#function to rgb images given an array with the corresponding classes fro each pixel

def get_image_from_classes(Y):
  
  colours={
    0:[255,255,255],                                    #white  ---> no class
    1:[0,0,0],                                          #black  ---> roads
    2:[100,100,100],                                    #grey   ---> buildings
    3:[0,0,150],                                        #dark blue  ---> water
    4:[0,255,0],                                        #light green  ---> grass
    5:[255,255,0],                                      #yellow   ---> rails
    6:[0,125,0],                                        #dark green  ---> trees
    7:[150,150,255],                                    #light blue  --->pools
    8:[150,80,0]                                        #brown  --->bare soil
    }
  
  temp=np.zeros((Y.shape[0],Y.shape[1],3))
  for j in range(Y.shape[0]):
    for k in range(Y.shape[1]):
      temp[j,k]=colours[Y[j,k]]
          
  return temp

In [ ]:
#normalising the array values

def normalize(img):
    min = img.min()
    max = img.max()
    x = 2.0 * (img - min) / (max - min) - 1.0
    return x

In [ ]:
#get pixelwise one hot encodings for each of the 9 classes from the rgb images 

def get_mask(img,Y):

    colours={
    0:[255,255,255],                                    #white  ---> no class
    1:[0,0,0],                                          #black  ---> roads
    2:[100,100,100],                                    #grey   ---> buildings
    3:[0,0,150],                                        #dark blue  ---> water
    4:[0,255,0],                                        #light green  ---> grass
    5:[255,255,0],                                      #yellow   ---> rails
    6:[0,125,0],                                        #dark green  ---> trees
    7:[150,150,255],                                    #light blue  ---> pools
    8:[150,80,0]                                        #brown  ---> bare soil
    }

 
    mask=np.zeros((img.shape[0],img.shape[1],9))
    for j in range(mask.shape[0]):
        for k in range(mask.shape[1]):
            for l in range(9):
              if colours[l]==list(Y[j,k]):
                mask[j,k,l]=1

    return mask

In [ ]:
#creating numpy arrays of the given sat and gt images 

def create_data():

    path="./"

    i=1

    X_train=[]
    Y_train=[]

    while True:
        sat_path=path+'sat/'+str(i)+'.tif'
        if os.path.exists(sat_path):
            img=np.array(imageio.imread(sat_path,format="tiff"))               #reading the satellite images 
            print("shape of image",i,":",img.shape,end=" ")
            img = resize(img)
            img = normalize(img)
            gt_path = path+'gt/'+str(i)+'.tif'
            mask = cv2.imread(gt_path)                                         #reading the ground truths
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)               
            mask = resize(mask)                                               
            mask = get_mask(img,mask)
            X_train.append(img)
            Y_train.append(mask)

        else:
            print("dataset ban gya....")
            print("ab chill!!")
            break

        i=i+1
    
    #saving the modified image data
    
    np.save("./rawdata_X_new.npy",X_train)                                     
    np.save("./rawdata_Y_new.npy",Y_train)
    data=np.load("./rawdata_Y_new.npy")
    data_x=np.load("./rawdata_X_new.npy")
    print(data[0].shape)
    print(data_x[0].shape)
    
    return np.array(X_train),np.array(Y_train)


In [ ]:
#function to calculate the frequency of each class
#used just a tool that might be helpful in determining the class weights used later (to determine the contribution of each class in the overall loss)



def class_frequency():
    a=[0,0,0,0,0,0,0,0,0]
    data=np.load("./rawdata_Y_new.npy")
    for i in  range(len(data)):
        temp= get_classes_from_image(data[i])
        img_height = temp.shape[0]
        img_width = temp.shape[1]
        print(temp[0,0])
        a[1] += np.sum(temp==1)
        a[2] += np.sum(temp==2)
        a[3] += np.sum(temp==3)
        a[4] += np.sum(temp==4)
        a[5] += np.sum(temp==5)
        a[6] += np.sum(temp==6)
        a[7] += np.sum(temp==7)
        a[8] += np.sum(temp==8)
    for i in range(10):
      print(a[i])
    return a

In [ ]:
#function to obtain the linear image patches using non-overlapping sliding window method

def get_linear_patch(X,Y,patch_size=patch_size):
    X_linear=[]
    Y_linear=[]
    patch_count=0
    print(X.shape,Y.shape)

    for i in range(X.shape[0]):
        img_x=X[i]
        img_y=Y[i]
        
        #determining no. of horizontal and vertical patches
        
        n_horizontal_patches=math.ceil(img_x.shape[0]/patch_size)
        n_vertical_patches=math.ceil(img_x.shape[1]/patch_size)
        print("no. of horizontal patches:",n_horizontal_patches,"    no. of vertical patches:",n_vertical_patches)
        print(img_x.shape,img_y.shape)
        
        for k in range(0,n_horizontal_patches):
            for l in range(0,n_vertical_patches):
              
              temp_x=img_x[k*patch_size:(k+1)*patch_size,l*patch_size:(l+1)*patch_size,:]
              temp_y=img_y[k*patch_size:(k+1)*patch_size,l*patch_size:(l+1)*patch_size,:]
              print("loading entry "+str(i)+"......")
              print(k*patch_size,(k+1)*patch_size,l*patch_size,(l+1)*patch_size)
              np.save("./Image_patch_new/img_patch"+str(patch_count)+".npy",temp_x)
              np.save("./mask_patch_new/mask_patch"+str(patch_count)+".npy",temp_y)
              patch_count=patch_count+1
    print(patch_count)      

In [ ]:
def transform(patch_img, patch_mask):
    # Apply some random transformations
    random_transformation = np.random.randint(1,8)
    if random_transformation == 1:  # reverse first dimension
        patch_img = patch_img[::-1,:,:]
        patch_mask = patch_mask[::-1,:,:]
    elif random_transformation == 2:    # reverse second dimension
        patch_img = patch_img[:,::-1,:]
        patch_mask = patch_mask[:,::-1,:]
    elif random_transformation == 3:    # transpose(interchange) first and second dimensions
        patch_img = patch_img.transpose([1,0,2])
        patch_mask = patch_mask.transpose([1,0,2])
    elif random_transformation == 4:
        patch_img = np.rot90(patch_img, 1)
        patch_mask = np.rot90(patch_mask, 1)
    elif random_transformation == 5:
        patch_img = np.rot90(patch_img, 2)
        patch_mask = np.rot90(patch_mask, 2)
    elif random_transformation == 6:
        patch_img = np.rot90(patch_img, 3)
        patch_mask = np.rot90(patch_mask, 3)
    else:
        pass

    return patch_img, patch_mask


In [ ]:
#generating random patches from the given dataset

def get_rand_patch(img, mask, sz=patch_size):
    """
    :param img: ndarray with shape (x_sz, y_sz, num_channels)
    :param mask: binary ndarray with shape (x_sz, y_sz, num_classes)
    :param sz: size of random patch
    :return: patch with shape (sz, sz, num_channels)
    """
    assert len(img.shape) == 3 and img.shape[0] > sz and img.shape[1] > sz and img.shape[0:2] == mask.shape[0:2]
    xc = random.randint(0, img.shape[0] - sz)
    yc = random.randint(0, img.shape[1] - sz)
    patch_img = img[xc:(xc + sz), yc:(yc + sz)]
    patch_mask = mask[xc:(xc + sz), yc:(yc + sz)]


    return transform(patch_img,patch_mask)

In [ ]:
def get_patches(X_train, Y_train , n_patches, patch_size=patch_size):
    X = []
    Y = []
    patch_count=0
    while patch_count < n_patches:
      
        img_id = random.randrange(0,X_train.shape[0])
        print("chosen image:",img_id)
        img = X_train[img_id]
        mask = Y_train[img_id]
        img_patch, mask_patch = get_rand_patch(img, mask, patch_size)
        np.save("./Image_patch_new/img_patch"+str(patch_count+232)+".npy",img_patch)
        np.save("./mask_patch_new/mask_patch"+str(patch_count+232)+".npy",mask_patch)
        patch_count += 1
    print(str(patch_count)+" patches generated...........")